<a href="https://colab.research.google.com/github/MahdiTheGreat/Intro-to-language-modeling/blob/main/intro_to_language_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/MahdiTheGreat/Intro-to-language-modeling.git
%cd Intro-to-language-modeling

Cloning into 'Intro-to-language-modeling'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 22 (delta 10), reused 2 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 28.29 MiB | 18.92 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/Intro-to-language-modeling


In [ ]:
import sklearn

In [ ]:
!pip install ipdb
!pip install -U spacy
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 99.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninst

In [ ]:
import spacy
import torch
import matplotlib.pyplot as plt
import ipdb
import numpy as np
import random
import pandas as pd


In [ ]:
%pdb on

Automatic pdb calling has been turned ON


In [ ]:
# Set random seed for reproducibility
def set_seed(seed=2024):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_seed(1998)

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu'))
print(f'Using device: {device}')

Using device: cpu


In [ ]:
dataset='lmdemo'
zip_file = f"{dataset}.zip"
!unzip -q $zip_file
!rm $zip_file

In [ ]:
text=open(f'{dataset}/train.txt','r',encoding='utf-8').read()

In [ ]:
import spacy
from collections import Counter

# Load spaCy model for tokenization
nlp = spacy.load("en_core_web_sm")

class VocabularyBuilder:
    def __init__(self, max_voc_size=None):
        self.max_voc_size = max_voc_size
        self.str_to_int = {}
        self.int_to_str = {}
        self.special_tokens = ["BEGINNING", "END", "UNKNOWN"]

    def build_vocabulary(self, text):

        tokens = []

        # Process each sentence in the text
        for sent in text:
            tokens.append(nlp("BEGINNING")[0])  # Add "BEGINNING" at the start of each sentence
            sent = nlp(sent)
            tokens.extend([token for token in sent])  # Add sentence tokens
            tokens.append(nlp("END")[0])  # Add "END" at the end of each sentence

        token_counter = Counter()
        for token in tokens:
         if not token.is_space:
             token_counter[token.text.lower()] += 1

        # Start vocabulary with special tokens
        for idx, token in enumerate(self.special_tokens):
            self.str_to_int[token] = idx
            self.int_to_str[idx] = token

        # Select the most common tokens, considering max_voc_size - len(special_tokens)
        if self.max_voc_size is None:
            max_words = len(token_counter) - len(self.special_tokens)
            self.max_voc_size = max_words + len(self.special_tokens)
        else:
         max_words = self.max_voc_size - len(self.special_tokens)
        most_common_tokens = token_counter.most_common(max_words)

        for idx, (token, _) in enumerate(most_common_tokens, start=len(self.special_tokens)):
            self.str_to_int[token] = idx
            self.int_to_str[idx] = token

    def get_token_id(self, token):
        # Return the integer ID for a given token
        return self.str_to_int.get(token.lower(), self.str_to_int["UNKNOWN"])

    def get_token_str(self, token_id):
        # Return the original token string for a given integer ID
        return self.int_to_str.get(token_id, "UNKNOWN")

    def add_special_tokens_to_text(self, text):
        """
        Tokenizes the text by sentence and adds special 'BEGINNING' and 'END' tokens
        around each sentence.

        Parameters:
        - text (str): The input text.

        Returns:
        - List[str]: A list of tokens with special 'BEGINNING' and 'END' tokens added.
        """


    def sanity_check(self):
        # Check vocabulary size
        assert len(self.str_to_int) <= self.max_voc_size, "Vocabulary size exceeds max_voc_size."

        # Check special tokens exist and are unique
        for token in self.special_tokens:
            assert token in self.str_to_int, f"Missing special token: {token}"

        # Check if highly frequent words are included and rare ones are not
        common_words = ["the", "and"]
        rare_words = ["cuboidal", "epiglottis"]

        for word in common_words:
            assert word in self.str_to_int, f"Common word '{word}' not in vocabulary."

        for word in rare_words:
            assert word not in self.str_to_int, f"Rare word '{word}' should not be in vocabulary."

        # Check that mapping back and forth works for a test word
        test_word = "The"
        token_id = self.get_token_id(test_word)
        assert self.get_token_str(token_id) == test_word.lower(), "Round-trip token mapping failed."

        print("Sanity check passed!")


/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
example_text = [
    "The quick brown fox jumps over the lazy dog.",
    "Python is an amazing programming language.",
    "I love learning new things every day.",
    "The weather is sunny and warm today.",
    "This is an example sentence for demonstration.",
    "Artificial intelligence is transforming the world.",
    "Machine learning models can learn from data.",
    "Reading books can expand your knowledge.",
    "She enjoys painting in her free time.",
    "Let's go for a walk in the park later."
]


In [ ]:
vocab_builder = VocabularyBuilder()
vocab_builder.build_vocabulary(example_text)

# Example mappings
print("str_to_int:", vocab_builder.str_to_int)
print("int_to_str:", vocab_builder.int_to_str)

# Convert a token to integer ID and back to string
token_id = vocab_builder.get_token_id("example")
print("Token ID for 'example':", token_id)
print("Original token from ID:", vocab_builder.get_token_str(token_id))

str_to_int: {'BEGINNING': 0, 'END': 1, 'UNKNOWN': 2, 'beginning': 3, '.': 4, 'end': 5, 'the': 6, 'is': 7, 'an': 8, 'learning': 9, 'for': 10, 'can': 11, 'in': 12, 'quick': 13, 'brown': 14, 'fox': 15, 'jumps': 16, 'over': 17, 'lazy': 18, 'dog': 19, 'python': 20, 'amazing': 21, 'programming': 22, 'language': 23, 'i': 24, 'love': 25, 'new': 26, 'things': 27, 'every': 28, 'day': 29, 'weather': 30, 'sunny': 31, 'and': 32, 'warm': 33, 'today': 34, 'this': 35, 'example': 36, 'sentence': 37, 'demonstration': 38, 'artificial': 39, 'intelligence': 40, 'transforming': 41, 'world': 42, 'machine': 43, 'models': 44, 'learn': 45, 'from': 46, 'data': 47, 'reading': 48, 'books': 49, 'expand': 50, 'your': 51, 'knowledge': 52, 'she': 53, 'enjoys': 54, 'painting': 55, 'her': 56, 'free': 57, 'time': 58, 'let': 59, "'s": 60, 'go': 61, 'a': 62}
int_to_str: {0: 'BEGINNING', 1: 'END', 2: 'UNKNOWN', 3: 'beginning', 4: '.', 5: 'end', 6: 'the', 7: 'is', 8: 'an', 9: 'learning', 10: 'for', 11: 'can', 12: 'in', 13: '

In [ ]:
# Perform sanity check
vocab_builder.sanity_check()

Sanity check passed!


In [ ]:
class TrainingDataPreparer:
    def __init__(self, vocab_builder, context_window_size):
        self.vocab_builder = vocab_builder
        self.N = context_window_size

    def encode_text(self, text):
        """Tokenizes and encodes a single string with special symbols.

        Parameters:
        - text (str): The input string to encode.

        Returns:
        - List[int]: A list of token IDs including BEGINNING and END tokens.
        """
        # Tokenize the text
        tokens = ["BEGINNING"] * self.N  # Add N BEGINNING tokens at the start

        for sent in text:
         tokens.append(nlp("BEGINNING")[0])  # Add "BEGINNING" at the start of each sentence
         sent = nlp(sent)
         tokens.extend([token for token in sent])  # Add sentence tokens
         tokens.append(nlp("END")[0])  # Add "END" at the end of each sentenc

        # Map tokens to integer IDs, using "UNKNOWN" for out-of-vocabulary words
        token_ids = [self.vocab_builder.get_token_id(str(token)) for token in tokens]
        return token_ids

    def create_training_sequences(self, text):
        """
        Creates training sequences from a single string by generating sequences of length N+1.

        Parameters:
        - text (str): The input string to create sequences from.

        Returns:
        - List[Tuple[List[int], int]]: A list of (context, target) pairs.
        """
        training_sequences = []

        # Encode the text with BEGINNING, END, and UNKNOWN tokens
        encoded_text = self.encode_text(text)

        # Generate sequences of length N+1
        for i in range(len(encoded_text) - self.N):
            context = encoded_text[i : i + self.N]  # N tokens for context
            target = encoded_text[i + self.N]       # Next token as the target
            training_sequences.append((context, target))

        return training_sequences


In [ ]:
context_window_size = 3
data_preparer = TrainingDataPreparer(vocab_builder, context_window_size)

# Tokenize text for training sequences
#paragraphs = [
#    ["this", "is", "a", "simple", "example", "sentence"],
#    ["here's", "another", "example", "sentence", "in", "a", "different", "paragraph"],
#    ["the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"]
#]

# Create training sequences
training_sequences = data_preparer.create_training_sequences(example_text)

# Display some training sequences
print("Training sequences (context, target):")
for context, target in training_sequences[:15]:  # Show the first few sequences
    print([vocab_builder.get_token_str(id) for id in context], "->", vocab_builder.get_token_str(target))

> <ipython-input-112-e2738e62edf1>(26)encode_text()
     25         ipdb.set_trace()
---> 26         token_ids = [self.vocab_builder.get_token_id(str(token)) for token in tokens]
     27         return token_ids

ipdb> tokens
['BEGINNING', 'BEGINNING', 'BEGINNING', BEGINNING, The, quick, brown, fox, jumps, over, the, lazy, dog, ., END, BEGINNING, Python, is, an, amazing, programming, language, ., END, BEGINNING, I, love, learning, new, things, every, day, ., END, BEGINNING, The, weather, is, sunny, and, warm, today, ., END, BEGINNING, This, is, an, example, sentence, for, demonstration, ., END, BEGINNING, Artificial, intelligence, is, transforming, the, world, ., END, BEGINNING, Machine, learning, models, can, learn, from, data, ., END, BEGINNING, Reading, books, can, expand, your, knowledge, ., END, BEGINNING, She, enjoys, painting, in, her, free, time, ., END, BEGINNING, Let, 's, go, for, a, walk, in, the, park, later, ., END]
ipdb> exit


In [ ]:
## Creating a tensor dataset ##
from torch.utils.data import DataLoader, TensorDataset
def TorchDataLoader(training_sequences, batch_size):
  context_words = [item[0] for item in training_sequences]  # List of [context]
  target_words = [item[1] for item in training_sequences]   # List of target words

  # Convert lists to tensors
  context_tensor = torch.tensor(context_words, dtype=torch.long)  # Shape: (num_samples, 3)
  target_tensor = torch.tensor(target_words, dtype=torch.long)    # Shape: (num_samples,)

  # Create a TensorDataset
  dataset = TensorDataset(context_tensor, target_tensor)

  # Create a DataLoader for batching
  batch_size = 4
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

  # Iterate over the DataLoader
  for batch_context, batch_target in dataloader:
      print("Batch context:", batch_context)
      print("Batch target:", batch_target)
      # You can now use batch_context and batch_target for model training

  return dataloader

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


class SimpleANN(nn.Module):

    def __init__(self,layer_sizes,activation=nn.ReLU,last_layer_activation=nn.ReLU,dropout=0):

        super(SimpleANN, self).__init__()
        self.layers = nn.ModuleList()

        for i in range(len(layer_sizes)-2):
          self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
          self.layers.append(nn.Dropout(dropout))
          self.layers.append(activation())

        self.layers.append(nn.Linear(layer_sizes[-2], layer_sizes[-1]))
        if last_layer_activation is not None:
         self.layers.append(nn.Dropout(dropout))
         self.layers.append(last_layer_activation())

    def forward(self, x):
        x = x.view(-1, np.prod(x.shape[1:])) # Flatten the input
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
from copy import deepcopy
def dense_arch_builder(input_size,scale_factor=0,hidden_layers_num=0,repeat=0,output_size=1):
  layer_sizes=[input_size]

  if scale_factor!=0:

   if scale_factor>1:
    for i in range(hidden_layers_num):
     layer_sizes.append(layer_sizes[-1]*scale_factor)
    while layer_sizes[-1]<output_size:
     layer_sizes.append(layer_sizes[-1]*scale_factor)

   elif scale_factor==1:
     for i in range(2,hidden_layers_num+2):
      layer_sizes.append(layer_sizes[0]*i)
     i+=1
     while layer_sizes[-1]<output_size:
      layer_sizes.append(layer_sizes[0]*i)
      i+=1

   mirrored_layer_sizes=deepcopy(layer_sizes)
   mirrored_layer_sizes.reverse()
   mirrored_layer_sizes=mirrored_layer_sizes[1:-1]

   for i in range(repeat):
    layer_sizes.append(layer_sizes[-1])

   if output_size>0:
    layer_sizes+=mirrored_layer_sizes
    downscale_factor=scale_factor if scale_factor>1 else 2

    while layer_sizes[-1]!=output_size:
     if layer_sizes[-1]//downscale_factor>=output_size:
      layer_sizes.append(layer_sizes[-1]//downscale_factor)
     else:
      layer_sizes.append(output_size)

  else:
    downscale_factor = (input_size / output_size) ** (1 / hidden_layers_num)
    for i in range(hidden_layers_num):
        layer_sizes.append(int(input_size / (downscale_factor ** i)))
    layer_sizes.append(output_size)

  return layer_sizes


In [ ]:
# Helper function to plot the training metrics

def plot_training_metrics(train_acc, val_acc, train_loss, title, save_path):
    # Ensure that all input lists have the same length
    assert len(train_acc) == len(val_acc) == len(train_loss), "All input histories must have the same length."

    epochs = range(1, len(train_acc) + 1)

    # Create the metrics DataFrame
    df_metrics = pd.DataFrame({
        'Epoch': epochs,
        'Training Accuracy (%)': train_acc,
        'Validation Accuracy (%)': val_acc,
        'Training Loss': train_loss
    })

    # Initialize the plot
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot Training and Validation Accuracy on ax1
    color = 'tab:blue'
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy (%)', color=color)
    ax1.plot(df_metrics['Epoch'], df_metrics['Training Accuracy (%)'], label='Train Acc', color='tab:blue')
    ax1.plot(df_metrics['Epoch'], df_metrics['Validation Accuracy (%)'], label='Val Acc', color='tab:cyan')
    ax1.tick_params(axis='y', labelcolor=color)

    # Create a second y-axis for Training Loss
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Loss', color=color)
    ax2.plot(df_metrics['Epoch'], df_metrics['Training Loss'], label='Train Loss', color='tab:red')
    ax2.tick_params(axis='y', labelcolor=color)

    # Combine legends from both axes
    lines_1, labels_1 = ax1.get_legend_handles_labels()
    lines_2, labels_2 = ax2.get_legend_handles_labels()
    ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper left')

    # Set plot title and layout
    plt.title(title)
    plt.tight_layout()

    # Save and display the plot
    plt.savefig(save_path)
    plt.show()